In [1]:
import pandas as pd

path_to_train = './data/train.csv'

train = pd.read_csv(path_to_train, encoding='utf-8', usecols=['genre_id', 'user_id', 'context_type', 'ts_listen', 'release_date', 'platform_name', 'platform_family', 'media_duration', 'listen_type', 'user_gender', 'is_listened'])

In [2]:
x_cols = ['genre_id', 'user_id', 'context_type', 'ts_listen', 'release_date', 'platform_name', 'platform_family', 'media_duration', 'listen_type', 'user_gender']
y_col = 'is_listened'

Цікава думка. Може треба тренуватися тільки на записах, котрі з Flow (listen_type=1)? Я думаю, так і треба робити.

In [3]:
# train-test-X-Y
import validation as v
train_i, validation_i = v.only_flow_split_train_validation_by_last_record(train)
Y_train = train.loc[train_i][y_col]
X_train = train.loc[train_i][x_cols]

Y_validation = train.loc[validation_i][y_col]
X_validation = train.loc[validation_i][x_cols]

In [4]:
# fit
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf = clf.fit(X_train, Y_train)

In [5]:
# validate
from sklearn.metrics import roc_auc_score
import numpy as np
Y_predicted = clf.predict_proba(X_validation)[:,1]
roc_auc_score(np.array(Y_validation), Y_predicted)

0.57716115422326397

In [11]:
# let's do the submission!
path_to_test = './data/test.csv'
path_to_submission = './data/submission.csv'

test = pd.read_csv(path_to_test, encoding='utf-8', usecols=['sample_id', 'genre_id', 'user_id', 'context_type', 'ts_listen', 'release_date', 'platform_name', 'platform_family', 'media_duration', 'listen_type', 'user_gender'],index_col='sample_id')

X_test = test[x_cols]
Y_test = clf.predict_proba(X_test)[:,1]

submission = pd.DataFrame(columns=['sample_id', 'is_listened'])
submission['is_listened'] = Y_test
submission['sample_id'] = test.index

submission.to_csv(path_to_submission, encoding='utf-8', index=False)

А якщо спробувати те саме, тільки взяти інші дані для тренування?

In [12]:
# train-test-X-Y
import validation as v
train_i, validation_i = v.split_train_validation_by_last_record_and_flow(train)
Y_train = train.loc[train_i][y_col]
X_train = train.loc[train_i][x_cols]

Y_validation = train.loc[validation_i][y_col]
X_validation = train.loc[validation_i][x_cols]

In [13]:
clf = LogisticRegression()
clf = clf.fit(X_train, Y_train)
Y_predicted = clf.predict_proba(X_validation)[:,1]
roc_auc_score(np.array(Y_validation), Y_predicted)

0.57716093554493164